<a href="https://colab.research.google.com/github/denniesbor/twitter_political_polarization/blob/from_colab/notebooks/data_collection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# !pip install selenium
# !pip install geopandas


In [2]:
from google.colab import drive

drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import re
import json
import time
import requests
import warnings
from datetime import datetime, date

# from selenium import webdriver
# from selenium.common.exceptions import TimeoutException
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.wait import WebDriverWait
# from bs4 import BeautifulSoup
# import tabula
import pandas as pd
import numpy as np
import geopandas
import seaborn as sns

warnings.filterwarnings("ignore")

In [4]:
# A python dictionary to abbr US states

us_state_to_abbrev = {
    "Alabama": "AL",
    "Alaska": "AK",
    "Arizona": "AZ",
    "Arkansas": "AR",
    "California": "CA",
    "Colorado": "CO",
    "Connecticut": "CT",
    "Delaware": "DE",
    "Florida": "FL",
    "Georgia": "GA",
    "Hawaii": "HI",
    "Idaho": "ID",
    "Illinois": "IL",
    "Indiana": "IN",
    "Iowa": "IA",
    "Kansas": "KS",
    "Kentucky": "KY",
    "Louisiana": "LA",
    "Maine": "ME",
    "Maryland": "MD",
    "Massachusetts": "MA",
    "Michigan": "MI",
    "Minnesota": "MN",
    "Mississippi": "MS",
    "Missouri": "MO",
    "Montana": "MT",
    "Nebraska": "NE",
    "Nevada": "NV",
    "New Hampshire": "NH",
    "New Jersey": "NJ",
    "New Mexico": "NM",
    "New York": "NY",
    "North Carolina": "NC",
    "North Dakota": "ND",
    "Ohio": "OH",
    "Oklahoma": "OK",
    "Oregon": "OR",
    "Pennsylvania": "PA",
    "Rhode Island": "RI",
    "South Carolina": "SC",
    "South Dakota": "SD",
    "Tennessee": "TN",
    "Texas": "TX",
    "Utah": "UT",
    "Vermont": "VT",
    "Virginia": "VA",
    "Washington": "WA",
    "West Virginia": "WV",
    "Wisconsin": "WI",
    "Wyoming": "WY",
    "District of Columbia": "DC",
    "American Samoa": "AS",
    "Guam": "GU",
    "Northern Mariana Islands": "MP",
    "Puerto Rico": "PR",
    "United States Minor Outlying Islands": "UM",
    "U.S. Virgin Islands": "VI",
    "Maine": "ME",
}

# dictionary of denominations

christian_categories = {
    "Christian": "Christian",
    "Christian - Roman Catholic": "Christian",
    "Christian - Baptist": "Christian",
    "Christian - Mormon": "Christian",
    "Hindu": "Hindu",
    "Jewish": "Jewish",
    "Christian - Catholic": "Christian",
    "Christian - Evangelical Protestant": "Christian",
    "Christian - Church of God": "Christian",
    "Unknown/Refused": "Unknown",
    "Unaffiliated": "Unaffiliated",
    "Christian - Presbyterian": "Christian",
    "Christian - Methodist": "Christian",
    "Christian - Lutheran": "Christian",
    "Christian - Episcopalian": "Christian",
    "Christian - Southern Baptist": "Christian",
    "Christian - Protestant": "Christian",
    "Christian - Restorationist": "Christian",
    "Christian - Disciples of Christ": "Christian",
    "Christian - Nondenominational": "Christian",
    "Unitarian Universalist": "Christian",
    "Christian - Eastern Orthodox": "Orthodox",
    "Christian - Evangelical Free Church": "Christian",
    "Christian - Pentecostal": "Christian",
    "Christian - Seventh-Day Adventist": "Christian",
    "Christian - Nondenominational Protestant": "Christian",
    "Buddhist - Soka Gakkai": "Buddhist",
    "Muslim": "Muslim",
    "Christian - African Methodist Episcopal": "Christian",
    "Christian - Chaldean Catholic": "Christian",
    "Christian - Congregationalist": "Christian",
    "Christian - United Brethren": "Christian",
    "Christian - Reformed (Calvinist)": "Christian",
    "Buddhist - Jodo Shinshu": "Buddhist",
    "Christian - United Church of Christ": "Christian",
    "Christian - Nazarene (Holiness)": "Christian",
    "uknown": "uknown",
}

# dictionary of ethnicities

ethnicity = {
    "Hispanic - Mexican": "Hispanic",
    "White - Non-Hispanic": "White",
    "Asian - Thai": "Asian",
    "Asian - Indian": "Asian",
    "Black - African-American": "Black",
    "Black - African-American/Asian - Filipino": "Black",
    "Hispanic": "Hispanic",
    "Hispanic - Puerto Rican": "Hispanic",
    "White - Portuguese-American": "White",
    "Native American - Chickasaw": "Other",
    "Black - Jamaican": "Black",
    "Hispanic - Mexican / Jewish": "Hispanic",
    "Hispanic - Cuban": "Hispanic",
    "Asian - Korean": "Asian",
    "Hispanic - Dominican": "Hispanic",
    "Black - Eritrean": "Black",
    "Black - African-American/Trinidadian": "Black",
    "Hispanic - Mexican/Colombian": "Hispanic",
    "Asian - Chinese": "Asian",
    "Asian - Japanese": "Asian",
    "Hispanic - Guatemalan": "Hispanic",
    "Asian - Taiwanese": "Asian",
    "Native American - Ho-Chunk": "Other",
    "White - Middle Eastern - Lebanese": "White",
    "White - Middle Eastern - Palestinian": "White",
    "Hispanic - New Mexican (Hispanos)": "Hispanic",
    "Native American - Cherokee": "Other",
    "uknown": "uknown",
    "Asian - Filipino/Chinese": "Asian",
    "Asian - Vietnamese": "Asian",
    "Hispanic - Ecuadorean": "Hispanic",
    "Pacific Islander - Samoan": "Other",
    "White - Middle Eastern - Palestinian/Syrian": "White",
    "Black - Somali": "Black",
    "Native American - Laguna Pueblo": "Other",
    "Black - African-American/Hispanic - Puerto Rican": "Black",
    "Black - African-American/Bermudan": "Black",
    "Hispanic - Texan (Hispanos)": "Hispanic",
    "Black - Jamaican/Asian - Indian": "Black",
}

## The Govtrack ideology scores downloaded from Govtrack.us
The 2020 report dates from Jan 3 -2019 to Jan 3 - 2021 whereas the 2022 report is dated Jan 3 - 2021 to Jan 3 - 2022. The Govtrack
report of the current legislators is yet to be released.

In [5]:
WORKING_DIR = (
    "/content/drive/MyDrive/summer-2023/twitter_political_polarization/data_analysis"
)

In [6]:
# read the 2020 reports and merge
df_2020_rep = pd.read_csv(
    f"{WORKING_DIR}/raw_data/govtrack-stats-2020-house-ideology.csv"
)
df_2020_sen = pd.read_csv(
    f"{WORKING_DIR}/raw_data/govtrack-stats-2020-senate-ideology.csv"
)

df_2020 = pd.concat([df_2020_rep, df_2020_sen])
df_2020.district = df_2020.district.fillna("Senator")
df_2020["period"] = 116
df_2020.district = df_2020.district.apply(lambda dis: dis if type(dis) == str else dis)
df_2020.rename(columns={"id": "govtrack_id"}, inplace=True)

# read the 2022 reports
df_2022_rep = pd.read_csv(
    f"{WORKING_DIR}/raw_data/govtrack-stats-2022-house-ideology.csv"
)
df_2022_sen = pd.read_csv(
    f"{WORKING_DIR}/raw_data/govtrack-stats-2022-senate-ideology.csv"
)

df_2022 = pd.concat([df_2022_rep, df_2022_sen])
df_2022["period"] = 117
df_2022.district = df_2022.district.fillna("Senator")
df_2022.district = df_2022.district.apply(lambda dis: dis if type(dis) == str else dis)
df_2022.rename(columns={"id": "govtrack_id"}, inplace=True)

# congress biographies from github.com/unitedstates/congress-legislators

congress_historical = pd.read_csv(f"{WORKING_DIR}/raw_data/legislators-historical.csv")
congress_current = pd.read_csv(f"{WORKING_DIR}/raw_data/legislators-current.csv")
congress_data = pd.concat([congress_historical, congress_current])

## Veterans in Congress - Data from (militarytimes.com)

In [7]:
def get_veterans(url):
    # using selenium to acquire veterans in the congress. Data source infogram

    # Create a new Chrome session
    driver = webdriver.Chrome()
    wait = WebDriverWait(driver, 20)

    # Load the web page
    driver.get(url)  # Wait for the page to fully load
    driver.maximize_window()

    wait.until(
        EC.visibility_of_element_located((By.CSS_SELECTOR, ".igc-table-container"))
    )
    # short pause added in order to make sure all the elements are loaded after we know the first element was loaded
    time.sleep(0.5)

    soup = BeautifulSoup(driver.page_source, "lxml")

    div = soup.select_one("div.igc-table-container")

    df = pd.read_html(str(div))[0]

    df["State"] = df.District.apply(
        lambda dis: dis.split(" ")[0] if dis != "Georgia" else "GA"
    )
    df["District"] = df.District.apply(
        lambda dis: "0"
        if len(dis.split(" ")) == 1 or dis.split(" ")[1] == "(House)"
        else str(dis.split(" ")[1])
    )
    df["District"] = df.District.apply(
        lambda dis: "Senator" if dis == "Sen" or dis == "Senate" else dis
    )
    df["Party"] = df.Party.apply(
        lambda party: "R"
        if party == "GOP" or party == "Republican"
        else "D"
        if party == "Dem"
        else party
    )

    return df

In [8]:
# df_mil_2020 = get_veterans(
#     "https://e.infogram.com/b28274cb-26b9-441f-9994-a8eb8558839b?src=embed"
# )
# df_mil_2020.to_csv("raw_data/veterans_in_congress_116.csv", index=False)

# df_mil_2022 = get_veterans(
#     "https://infogram.com/veterans-in-117th-congress-1hxj48ppv9zz52v"
# )
# df_mil_2022.to_csv("raw_data/veterans_in_congress_117.csv", index=False)

## Other Congress biographies and social links - source https://github.com/unitedstates/congress-legislators

In [9]:
congress_files_dir = "/content/drive/MyDrive/summer-2023"

In [10]:
df_senate = pd.read_csv(
    f"{congress_files_dir}/116th Congress Members Guide with Elections and Demographic Data by District - Senate.csv"
)
df_house = pd.read_csv(
    f"{congress_files_dir}/116th Congress Members Guide with Elections and Demographic Data by District - House.csv"
)

# senators in the military in the 116th congress
df_vets = pd.read_csv(f"{congress_files_dir}/military_2020.csv")
df_vets_house = df_vets[~(df_vets["District"] == "Senator")]
df_vets_senate = df_vets[df_vets["District"] == "Senator"]


# drop the first two rows
df_house = df_house.iloc[2:]
df_house.reset_index(inplace=True, drop=True)
df_senate = df_senate.iloc[2:]
df_senate.reset_index(inplace=True, drop=True)

In [11]:
df_senate["period"] = 116
df_house["period"] = 116

In [12]:
df_senate["State"] = df_senate.State.apply(lambda state: us_state_to_abbrev[state])

In [13]:
# do some little clean up

df_house["Distr"] = df_house.Code.apply(
    lambda code: 0 if code.split("-")[1] == "AL" else int(code.split("-")[1])
)
df_house["State"] = df_house.Code.apply(lambda code: code.split("-")[0])

df_house.drop(["District", "Code"], axis=1, inplace=True)

In [14]:
df_vets_house["District"] = df_vets_house.District.apply(int)
# df_vets_senate["District"] = df_vets_senate.District.apply(int)

In [15]:
# merge df_house with df_vets using name and state
df_vets_house_names = df_vets_house["Name"].str.split(expand=True)

# Check if either first name or second name from df1 is in df2's name column
df_house["is_in_df2"] = (
    df_house["First Name"].isin(df_vets_house_names[0])
    | df_house["Last Name"].isin(df_vets_house_names[0])
    | df_house["First Name"].isin(df_vets_house_names[1])
    | df_house["Last Name"].isin(df_vets_house_names[1])
)
is_vet_house = df_house[df_house["is_in_df2"]]

df_house.drop("is_in_df2", axis=1, inplace=True)
merged_df_vet_house = df_vets_house.merge(
    is_vet_house[["First Name", "Last Name", "Distr", "State"]],
    left_on=["District", "State"],
    right_on=["Distr", "State"],
    how="left",
)
merged_df_vet_house.rename(columns={"Service": "Military Service"}, inplace=True)

# merge house with the veteran df
df_house = df_house.merge(
    merged_df_vet_house[["First Name", "Last Name", "Distr", "Military Service"]],
    on=["First Name", "Last Name", "Distr"],
    how="left",
)
df_house.rename(columns={"Military Service_x": "Military Service"}, inplace=True)

df_house["Military Service"] = df_house["Military Service"].fillna("no")
df_house["is_vet"] = df_house["Military Service"].apply(
    lambda x: "No" if x == "no" else "Yes"
)
df_house.drop_duplicates(
    subset=["State", "Party", "First Name", "Last Name", "Distr"], inplace=True
)

# senate df

df_vets_senate_names = df_vets_senate["Name"].str.split(expand=True)

# Check if either first name or second name from df1 is in df2's name column
df_senate["is_in_df2"] = (
    df_senate["First Name"].isin(df_vets_senate_names[0])
    | df_senate["Last Name"].isin(df_vets_senate_names[0])
    | df_senate["First Name"].isin(df_vets_senate_names[1])
    | df_senate["Last Name"].isin(df_vets_senate_names[1])
)

is_vet_senate = df_senate[df_senate["is_in_df2"]]

df_senate.drop("is_in_df2", axis=1, inplace=True)
merged_df_vet_senate = df_vets_senate.merge(
    is_vet_senate[["First Name", "Last Name", "State"]],
    left_on=["State"],
    right_on=["State"],
    how="left",
)
merged_df_vet_senate.rename(columns={"Service": "Military Service"}, inplace=True)

# merge senate df with the veteran df
df_senate = df_senate.merge(
    merged_df_vet_senate[["First Name", "Last Name", "Military Service", "State"]],
    on=["First Name", "Last Name", "State"],
    how="left",
)
df_senate.rename(columns={"Military Service_x": "Military Service"}, inplace=True)

df_senate["Military Service"] = df_senate["Military Service"].fillna("no")
df_senate["is_vet"] = df_senate["Military Service"].apply(
    lambda x: "No" if x == "no" else "Yes"
)
df_senate.drop_duplicates(
    subset=["State", "Party", "First Name", "Last Name"], inplace=True
)

In [16]:
df_senate["Median Income"] = (
    df_senate["Median Income"]
    .replace({"\$": "", ",": "", " ": ""}, regex=True)
    .astype(int)
)
df_house["Median Income"] = (
    df_house["Median Income"]
    .replace({"\$": "", ",": "", " ": ""}, regex=True)
    .astype(int)
)

df_senate["White Income"] = (
    df_senate["White Income"]
    .replace({"\$": "", ",": "", " ": ""}, regex=True)
    .astype(int)
)
df_house["White Income"] = (
    df_house["White Income"]
    .replace({"\$": "", ",": "", " ": ""}, regex=True)
    .astype(int)
)

In [17]:
df_house.rename(columns={"Distr": "District"}, inplace=True)
df_senate.rename(columns={"Class": "Senate Class"}, inplace=True)

df_house.Gender = df_house.Gender.apply(
    lambda gender: "M" if gender == "Man" else "F" if gender == "Woman" else gender
)
df_senate.Gender = df_senate.Gender.apply(
    lambda gender: "M" if gender == "Man" else "F" if gender == "Woman" else gender
)

In [18]:
# us border states
border_canada = [
    "Alaska",
    "Michigan",
    "Maine",
    "Minnesota",
    "Montana",
    "New York",
    "Washington",
    "North Dakota",
    "Ohio",
    "Vermont",
    "New Hampshire",
    "Idaho",
]

border_mexico = ["Texas", "Arizona", "New Mexico", "California"]

ca_border = [us_state_to_abbrev[state] for state in border_canada]
me_border = [us_state_to_abbrev[state] for state in border_mexico]

# add border states
df_senate["is_border"] = df_senate.State.apply(
    lambda state: "canada_border"
    if state in ca_border
    else "mexico_border"
    if state in me_border
    else "no"
)

# add border states
df_house["is_border"] = df_house.State.apply(
    lambda state: "canada_border"
    if state in ca_border
    else "mexico_border"
    if state in me_border
    else "no"
)

df_house.columns = [
    (col.replace("/", "").replace("-", " ")).replace(" ", "_").lower()
    for col in df_house.columns
]
df_senate.columns = [
    (col.replace("/", "").replace("-", " ")).replace(" ", "_").lower()
    for col in df_senate.columns
]

In [19]:
congress_member_data = [
    "party",
    "first_name",
    "last_name",
    "elected_appointed",
    "birth_year",
    "gender",
    "race_ethnicity",
    "religion",
    "lgbtq",
    "pre_election_incumbent",
    "period",
    "military_service",
    "is_vet",
]

house_bio_data = df_house[congress_member_data + ["district", "state"]]
senate_bio_data = df_senate[congress_member_data + ["senate_class", "year", "state"]]
senate_bio_data["senate_class"] = senate_bio_data.senate_class.apply(
    lambda x: dict(zip(["II", "III", "I"], [2, 3, 1]))[x]
)

congressional_district_demographic_data = df_house.drop(congress_member_data, axis=1)
state_demographic_data = df_senate.drop(
    congress_member_data + ["senate_class", "year"], axis=1
)

state_demographic_data.drop_duplicates(inplace=True)
state_demographic_data.reset_index(inplace=True, drop=True)

# state_demographic_data['population_white_percentage'] = (state_demographic_data['2014_2018_acs_citizen_adult_population_white'] / state_demographic_data[['2014_2018_acs_citizen_adult_population_white',
#        '2014_2018_acs_citizen_adult_population_black',
#        '2014_2018_acs_citizen_adult_population_latino',
#        '2014_2018_acs_citizen_adult_population_asian_and_pacific_islander',
#        '2014_2018_acs_citizen_adult_population_native',
#        '2014_2018_acs_citizen_adult_population_other',]].sum(axis=1))*100

# congressional_district_demographic_data['population_white_percentage'] = (congressional_district_demographic_data['2014_2018_acs_citizen_adult_population_white'] / congressional_district_demographic_data[['2014_2018_acs_citizen_adult_population_white',
#        '2014_2018_acs_citizen_adult_population_black',
#        '2014_2018_acs_citizen_adult_population_latino',
#        '2014_2018_acs_citizen_adult_population_asian_and_pacific_islander',
#        '2014_2018_acs_citizen_adult_population_native',
#        '2014_2018_acs_citizen_adult_population_other',]].sum(axis=1))*100

congressional_district_demographic_data.to_csv(
    "/content/drive/MyDrive/summer-2023/raw_data/broadband/processed_data/congressional_district_demographic_data.csv",
    index=False,
)
state_demographic_data.to_csv(
    "/content/drive/MyDrive/summer-2023/raw_data/broadband/processed_data/state_demographic_data.csv",
    index=False,
)

In [20]:
# cluster ideologies
def clusters(value):
    if value <= 0.2:
        cluster = "Far Left"
    elif value > 0.2 and value <= 0.4:
        cluster = "Left Centrist"
    elif value > 0.4 and value <= 0.6:
        cluster = "Centrist"
    elif value > 0.6 and value <= 0.8:
        cluster = "Right Centrist"
    else:
        cluster = "Far Right"

    return cluster


# convert birth year string to date time object
def birthday(df) -> pd.DataFrame:
    df.birthday = pd.to_datetime(df.birthday)
    df["birth_year"] = df.birthday.dt.year

    return df


# calculate years in service
def since_elected(df):
    today = date.today()
    df["elected_appointed"] = pd.to_datetime(df["elected_appointed"])
    df["since_first_elected"] = df["elected_appointed"].apply(
        lambda x: today.year - x.year
    )

    return df


# compute the age of a congressman
def age(born):
    try:
        today = date.today()
        return (
            today.year - born.year - ((today.month, today.day) < (born.month, born.day))
        )
    except (ValueError, TypeError):
        return born  # leave unchanged


#  collapse religion and ethnicity
def collapse_religion_ethnicities(df):
    df["religion"] = df.religion.fillna("uknown")
    df["race_ethnicity"] = df.race_ethnicity.fillna("uknown")

    df["religion"] = df.religion.apply(
        lambda x: christian_categories[x] if x != "nan" else "uknown"
    )
    df["race_ethnicity"] = df.race_ethnicity.apply(
        lambda x: ethnicity[x] if x else "uknown"
    )

    df["christian"] = df.religion.apply(
        lambda religion: "Yes" if religion == "Christian" else "No"
    )
    df["lgbtq"] = df.lgbtq.apply(lambda lgbtq: "No" if lgbtq == "No" else "Yes")

    df["party"] = df.party.apply(
        lambda x: "Democratic" if x == "Independent - Dem Caucus" else x
    )

    # # add since the rep was elected
    df = since_elected(df)

    # # add ideological cluster
    df["ideology_cluster"] = df.ideology.apply(clusters)

    return df


congress_data = birthday(congress_data)
congress_data["age"] = congress_data.birthday.apply(age)
congress_data.columns = [col.replace(" ", "_") for col in congress_data.columns]
congress_data["district"] = congress_data["district"].fillna(-2).apply(int)
congress_data_reduced = congress_data[
    [
        "last_name",
        "first_name",
        "birthday",
        "gender",
        "type",
        "state",
        "district",
        "senate_class",
        "party",
        "twitter",
        "twitter_id",
        "govtrack_id",
        "birth_year",
        "age",
    ]
]

In [21]:
# Join the historical and bios data of the congress members


def merge_hist_bios(df, chamber):
    df["birth_year"] = df["birth_year"].fillna(0).apply(int)

    # df.party = df.party.replace({"Democratic": "Democrat", 'Independent - Dem Caucus': "Independent"})

    df.party = df.party.apply(
        lambda party: "R"
        if party == "Republican"
        else "D"
        if party == "Democrat"
        else "I"
        if party == "Independent"
        else party
    )

    if chamber == "lower":
        on = ["last_name", "state", "gender", "district"]

    elif chamber == "upper":
        on = ["last_name", "state", "gender", "senate_class"]
    else:
        return

    filtered_columns = [
        col
        for col in congress_data_reduced.columns
        if col != "party" and col != "first_name"
    ]

    df = df.merge(congress_data_reduced[filtered_columns], on=on)

    df = df[~df.govtrack_id.isna()]

    # merge the new dataframe with the govtract scores using govtrack ids
    df = df.merge(df_2020[["ideology", "govtrack_id"]], on="govtrack_id")

    # df = df[~df.twitter.isna()]

    # collapse the religions
    df = collapse_religion_ethnicities(df)

    return df

In [22]:
senate_bio_data

,party,first_name,last_name,elected_appointed,birth_year,gender,race_ethnicity,religion,lgbtq,pre_election_incumbent,period,military_service,is_vet,senate_class,year,state
0,Democratic,Doug,Jones,2017,1954,M,White - Non-Hispanic,Christian - Methodist,No,Lost in primary,116,no,No,2,2017,AL
1,Republican,Richard,Shelby,1986,1934,M,White - Non-Hispanic,Christian - Presbyterian,No,Not up,116,no,No,3,2016,AL
2,Republican,Dan,Sullivan,2014,1964,M,White - Non-Hispanic,Christian - Roman Catholic,No,Not up,116,Army,Yes,2,2014,AK
4,Republican,Lisa,Murkowski,2002,1957,F,White - Non-Hispanic,Christian - Roman Catholic,No,Not up,116,no,No,3,2016,AK
5,Democratic,Kyrsten,Sinema,2018,1976,F,White - Non-Hispanic,Unaffiliated,Bisexual,Open - Retired,116,no,No,1,2018,AZ
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,Republican,Shelley Moore,Capito,2014,1953,F,White - Non-Hispanic,Christian - Presbyterian,No,Not up,116,no,No,2,2014,WV
97,Democratic,Tammy,Baldwin,2012,1962,F,White - Non-Hispanic,Unknown/Refused,Lesbian,Won,116,no,No,1,2018,WI
98,Republican,Ron,Johnson,2010,1955,M,White - Non-Hispanic,Christian - Lutheran,No,Not up,116,no,No,3,2016,WI
99,Republican,John,Barrasso,2007,1952,M,White - Non-Hispanic,Christian - Presbyterian,No,Won,116,no,No,1,2018,WY


In [23]:
cd116_bio = merge_hist_bios(house_bio_data, "lower")
congress116_senate_bio = merge_hist_bios(senate_bio_data, "upper")

# save the bio data of 116th congressmen
cd116_bio.to_csv(
    "/content/drive/MyDrive/summer-2023/raw_data/broadband/processed_data/cd116_bio.csv",
    index=False,
)
congress116_senate_bio.to_csv(
    "/content/drive/MyDrive/summer-2023/raw_data/broadband/processed_data/cs116_bio.csv",
    index=False,
)

### Twitter data and merge with the Biography data

In [24]:
# # read Twitter data

# twitter_data = pd.read_csv(
#     f"/content/drive/MyDrive/summer-2023/raw_data/raw_tweets.csv",
#     usecols=["username", "tweet", "created_at"],
# )


# twitter_data.username.replace(
#     {"SenMarcoRubio": "senRubioPress", "SenatorHeinrich": "MartinHeinrich"},
#     inplace=True,
# )


# def merge_twitter_congress_df(df):
#     # df.username.replace({"SenMarcoRubio": "senRubioPress", "SenatorHeinrich": "MartinHeinrich"}, inplace=True)
#     df = twitter_data.merge(
#         df,
#         left_on=twitter_data["username"].str.lower(),
#         right_on=df["twitter"].str.lower(),
#         how="left",
#     )

#     df.Party = df.Party.apply(
#         lambda party: "R"
#         if party == "Republican"
#         else "D"
#         if party == "Democratic"
#         else "I"
#         if party == "Independent"
#         else party
#     )

#     df = df[~df.ideology.isna()]

#     # Drop duplicate 'Name' columns and reset the index
#     df = df.drop(["key_0"], axis=1).reset_index(drop=True)

#     return df

In [25]:
# house = merge_twitter_congress_df(house)
# senate = merge_twitter_congress_df(senate)
# df = merge_twitter_congress_df(df)

In [26]:
# save the two dataframes

# house.to_csv("/content/drive/MyDrive/summer-2023/raw_data/house.csv", index=False)
# senate.to_csv("/content/drive/MyDrive/summer-2023/raw_data/senate.csv", index=False)

# df[['age', "username", "tweet", "created_at", 'ideology', 'id', 'Christian', 'is_border','2014_2018_ACS_education_high_school_or_greater','LGBTQ',
#        'since_first_elected', 'ideology_cluster',"govtrack_id",'state', 'district',"Party", "median_income", 'Elected/ Appointed', 'Birth Year', 'Gender', 'ethnicity', 'religion']].to_csv("/content/drive/MyDrive/summer-2023/raw_data/reduced_all_data.csv", index=False)

### Broadband datasets

In [27]:
from numpy import int32
from io import StringIO
import numpy as np

# loading the zcta to rural/urban crosswalks
# loading the zcta state_dist relationship files
with open(
    "/content/drive/MyDrive/summer-2023/raw_data/broadband/missouri_zcta_state_dist.csv",
    "rb",
) as file:
    df_st_cd_zcta = file.read().decode("utf-8", errors="replace")
df_st_cd_zcta = pd.read_csv(StringIO(df_st_cd_zcta))

with open(
    "/content/drive/MyDrive/summer-2023/raw_data/broadband/missouri_zcta_urban_rural.csv",
    "rb",
) as file:
    df_urban_rural = file.read().decode("utf-8", errors="replace")
df_urban_rural = pd.read_csv(StringIO(df_urban_rural))


def check_if_int(zip):
    try:
        return int(zip)
    except ValueError:
        # print(zip)
        return 0


df_urban_rural.zcta = df_urban_rural.zcta.apply(check_if_int)
df_st_cd_zcta.zcta = df_st_cd_zcta.zcta.apply(check_if_int)
# df_st_cd_zcta.zcta = df_st_cd_zcta.zcta.astype(int32)

# Some of the state-zip data consists of multiple congressional districts within demarcating a single zcta area
# Group by 'zcta' and calculate sum of 'population'
grouped = df_st_cd_zcta.groupby("zcta")["population_20"].sum().reset_index()

# Find the row with the highest population_20 within each group
max_pop_row_idx = df_st_cd_zcta.groupby("zcta")["population_20"].idxmax()

df_st_cd_zcta = df_st_cd_zcta.iloc[max_pop_row_idx]
df_st_cd_zcta.reset_index(inplace=True, drop=True)

# Update the 'population_20' column with the sum of population_20s
df_st_cd_zcta["population_20"] = grouped["population_20"].to_list()

# merge the two dataframes
df_ub_r = df_urban_rural.merge(
    df_st_cd_zcta[["zcta", "cd116", "state", "state_code"]], on=["zcta"], how="left"
)

In [28]:
columns = {
    "geographic area name": "zcta",
    "estimate_total_total households": "estimate_total_households",
    "estimate_total_total households_type of internet subscriptions_with an internet subscription:": "estimate_households_with_internet",
    "estimate_total_total households_type of internet subscriptions_with an internet subscription:_broadband of any type": "estimate_households_with_internet_broadband_any",
    "estimate_total_total households_type of internet subscriptions_without an internet subscription": "estimate_households_without_internet",
    # "estimate_percent_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_less than $20,000:",
    "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_less than $20,000:_with a broadband internet subscription": "estimate_households_income_$20000_less_with_internet",
    "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_less than $20,000:_without an internet subscription": "households_income_$20000_less_without_internet",
    # "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$20,000 to $74,999:",
    "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$20,000 to $74,999:_with a broadband internet subscription": "estimate_households_income_between_$20000_$74,999_with_internet",
    "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$20,000 to $74,999:_without an internet subscription": "estimate_households_income_between_$20000_$74,999_without_internet",
    # "estimate_percent_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$75,000 or more:",
    "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$75,000 or more:_with a broadband internet subscription": "estimate_households_income_$75,000_more_with_internet",
    "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$75,000 or more:_without an internet subscription": "estimate_households_income_$75,000_more_without_internet",
}

In [29]:
# read household broadband data per zip code.
# the data is based on ACS 2021 five year survey

df_acs_2021_broadband = pd.read_csv(
    "/content/drive/MyDrive/summer-2023/raw_data/broadband/broadband/2021_ZCTA_acs_household_broadband_connection_raw.csv"
)

df_acs_2021_broadband.columns = [
    (column.lower()).replace("!!", "_") for column in df_acs_2021_broadband.columns
]
df_acs_2021_broadband = df_acs_2021_broadband[
    [
        "geography",
        "geographic area name",
        "estimate_total_total households",
        "estimate_total_total households_type of internet subscriptions_with an internet subscription:",
        "estimate_total_total households_type of internet subscriptions_with an internet subscription:_broadband of any type",
        "estimate_total_total households_type of internet subscriptions_without an internet subscription",
        # "estimate_percent_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_less than $20,000:",
        "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_less than $20,000:_with a broadband internet subscription",
        "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_less than $20,000:_without an internet subscription",
        # "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$20,000 to $74,999:",
        "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$20,000 to $74,999:_with a broadband internet subscription",
        "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$20,000 to $74,999:_without an internet subscription",
        # "estimate_percent_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$75,000 or more:",
        "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$75,000 or more:_with a broadband internet subscription",
        "estimate_total_total households_household income in the past 12 months (in 2021 inflation-adjusted dollars)_$75,000 or more:_without an internet subscription",
    ]
]

df_acs_2021_broadband.rename(columns=columns, inplace=True)

df_acs_2021_broadband["zcta"] = (
    df_acs_2021_broadband["zcta"].apply(lambda zcta: zcta.split(" ")[1]).astype(int)
)
df_acs_2021_broadband.reset_index(inplace=True, drop=True)

In [30]:
broadband_household = df_ub_r.merge(df_acs_2021_broadband, on="zcta", how="left")

# drop the nans in the dataframe (zcta in the urb_rural not available in the broadband dataset)

broadband_household = broadband_household[~broadband_household.geography.isna()]
broadband_household.reset_index(inplace=True, drop=True)

In [31]:
# Step 1: Group by "state," "congressional_district," and "zcta_code"


def compute_broadband_percs(df):
    dfs_perc_cd = []
    dfs_perc_st = []

    for column in list(df.columns)[9:]:
        total_households = "estimate_total_households"
        broadband_var = column

        cd_level = "(cd) perc_" + broadband_var
        state_level = "(state) perc_" + broadband_var

        grouped = df.groupby(["state", "cd116"])[total_households, broadband_var].sum()
        grouped = grouped.reset_index()
        grouped[cd_level] = (grouped[broadband_var] / grouped[total_households]) * 100

        dfs_perc_cd.append(grouped[[cd_level, "state", "cd116"]])

        grouped = grouped.groupby("state").sum()
        grouped = grouped.reset_index()
        grouped[state_level] = (
            grouped[broadband_var] / grouped[total_households]
        ) * 100

        dfs_perc_st.append(grouped[[state_level, "state"]])

    df_percs_distr = pd.concat(dfs_perc_cd, axis=1)
    df_percs_st = pd.concat(dfs_perc_st, axis=1)

    _, i = np.unique(df_percs_distr.columns, return_index=True)
    df_percs_distr = df_percs_distr.iloc[:, i]

    _, i = np.unique(df_percs_st.columns, return_index=True)
    df_percs_st = df_percs_st.iloc[:, i]

    return df_percs_st, df_percs_distr

In [32]:
st_broadband_vars, cd_broadband_vars = compute_broadband_percs(
    broadband_household.drop_duplicates(subset="zcta")
)

### Classifying rural/urban population at state or district level

In [33]:
# compute the percentage of rural to total population in a state or region:

# Step 1: Group by "state," "congressional_district," and "zcta_code"
grouped = broadband_household.groupby(["state", "cd116", "urban_rural"])[
    "population_20"
].sum()

cd_urban_ratio = grouped.unstack(level="urban_rural")
cd_urban_ratio["cd_urban_pop_ratio"] = cd_urban_ratio["U"] / cd_urban_ratio.sum(axis=1)
cd_urban_ratio["total_district_population"] = cd_urban_ratio.sum(axis=1).round(0)
cd_urban_ratio["total_district_population"] = cd_urban_ratio[
    "total_district_population"
].round(0)

# Print the resulting DataFrame with the U/total population ratio
cd_urban_ratio.reset_index(inplace=True)
cd_urban_ratio = cd_urban_ratio[
    ["state", "cd116", "cd_urban_pop_ratio", "total_district_population"]
]

# merge with household broadband data

cd_urban_rural_demand = cd_urban_ratio.merge(cd_broadband_vars, on=["state", "cd116"])
cd_urban_rural_demand

,state,cd116,cd_urban_pop_ratio,total_district_population,(cd) perc_estimate_households_income_$20000_less_with_internet,"(cd) perc_estimate_households_income_$75,000_more_with_internet","(cd) perc_estimate_households_income_$75,000_more_without_internet","(cd) perc_estimate_households_income_between_$20000_$74,999_with_internet","(cd) perc_estimate_households_income_between_$20000_$74,999_without_internet",(cd) perc_estimate_households_with_internet,(cd) perc_estimate_households_with_internet_broadband_any,(cd) perc_estimate_households_without_internet,(cd) perc_households_income_$20000_less_without_internet
0,AK,0,0.650101,732158.0,6.563555,50.868898,2.413264,30.972137,5.397989,88.632561,88.404590,11.367439,3.556187
1,AL,1,0.662922,730653.0,10.680373,33.155781,2.112127,36.979452,8.570060,82.148339,80.815605,17.851661,7.169474
2,AL,2,0.505566,687953.0,11.641071,30.988098,2.178899,37.907575,8.904662,80.798646,80.536744,19.201354,8.117794
3,AL,3,0.523418,746063.0,12.109415,32.848725,1.928726,36.972575,8.326555,82.148096,81.930715,17.851904,7.596623
4,AL,4,0.328704,708017.0,10.914158,30.829032,2.173213,37.762185,10.429455,79.678430,79.505375,20.321570,7.718902
...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,WI,8,0.595225,751594.0,6.712425,43.107245,1.805595,37.261192,7.242963,87.509474,87.080861,12.490526,3.441967
433,WV,1,0.502263,599996.0,11.821566,32.971613,1.937403,37.956119,8.846834,83.109078,82.749298,16.890922,6.106686
434,WV,2,0.475610,620899.0,10.225646,34.828229,2.050283,37.383720,8.826610,82.789809,82.437596,17.210191,6.333298
435,WV,3,0.362838,572498.0,14.760878,24.376127,2.234368,37.777032,10.983175,77.204765,76.914036,22.795235,9.577693


In [34]:
grouped = broadband_household.groupby(["state", "urban_rural"])["population_20"].sum()

# Calculate the ratio of urban population (U) to the total population
urban_population_ratio = grouped.unstack(level="urban_rural")
urban_population_ratio["state_urban_pop_ratio"] = urban_population_ratio[
    "U"
] / urban_population_ratio.sum(axis=1)
urban_population_ratio["total_state_population"] = urban_population_ratio.sum(axis=1)
urban_population_ratio["total_state_population"] = urban_population_ratio[
    "total_state_population"
].round(0)

# Print the resulting DataFrame with the U/total population ratio
urban_population_ratio.reset_index(inplace=True)
state_urban_ratio_df = urban_population_ratio[
    ["state", "state_urban_pop_ratio", "total_state_population"]
]

# concatenate state_urban with broadband (households data)

state_urban_rural_demand = st_broadband_vars.merge(state_urban_ratio_df, on="state")


# broadband_demand

In [35]:
state_gdp = pd.read_csv(
    "/content/drive/MyDrive/summer-2023/raw_data/broadband/us_gdp_per_state.csv"
)

state_gdp = state_gdp[
    [
        "state",
        "current_dollar_gdp_2021",
        "current_dollar_gdp_2022",
        "real_gdp_2021",
        "real_gdp_2022",
    ]
].dropna()

state_gdp["state_code"] = state_gdp.state.apply(lambda x: us_state_to_abbrev[x.strip()])

for column in state_gdp.columns:
    state_gdp[column] = state_gdp[column].str.replace(",", "")

    try:
        state_gdp[column] = state_gdp[column].astype(int)
        state_gdp[column] = state_gdp[column] * 1000000
    except:
        pass

# merge the state gdp with the state_urban_ rural demand
state_ur_demand = state_urban_rural_demand.merge(
    state_gdp[
        [
            "state_code",
            "current_dollar_gdp_2021",
            "current_dollar_gdp_2022",
            "real_gdp_2021",
            "real_gdp_2022",
        ]
    ],
    left_on="state",
    right_on="state_code",
)

In [36]:
# 2022 real_gdp
state_ur_demand["2022_nominal_gdp_per_capita"] = (
    state_ur_demand["current_dollar_gdp_2022"]
    / state_ur_demand["total_state_population"]
).round(0)
state_ur_demand["2022_real_gdp_per_capita"] = (
    state_ur_demand["real_gdp_2022"] / state_ur_demand["total_state_population"]
).round(0)

state_ur_demand["2021_nominal_gdp_per_capita"] = (
    state_ur_demand["current_dollar_gdp_2021"]
    / state_ur_demand["total_state_population"]
).round(0)
state_ur_demand["2021_real_gdp_per_capita"] = (
    state_ur_demand["real_gdp_2021"] / state_ur_demand["total_state_population"]
).round(0)

In [37]:
# merge the entire dfs
broadband_demand = cd_urban_rural_demand.merge(state_ur_demand, on="state", how="left")

In [38]:
import pandas as pd
import gc
import os
import warnings

warnings.filterwarnings("ignore")

from tqdm import tqdm

tqdm.pandas(desc="My bar!")

file_path = "/content/drive/MyDrive/summer-2023/raw_data/broadband/broadband/"

if os.path.exists(f"{file_path}fcc_coverage.csv"):
    df_coverage = pd.read_csv(f"{file_path}fcc_coverage.csv")

else:
    chunksize = 100000
    df_chunks = []

    for chunk in tqdm(
        pd.read_csv(
            f"{file_path}fbd_us_with_satellite_dec2021_v1.csv",
            chunksize=chunksize,
            encoding="utf-8",
            usecols=[
                "Provider_Id",
                "BlockCode",
                "TechCode",
                "MaxAdDown",
                "MaxAdUp",
                "StateAbbr",
            ],
            **{"encoding_errors": "ignore"},
        )
    ):
        chunk["BlockCode"] = chunk.BlockCode.astype("int64")

        df_chunks.append(chunk)

    # concat the chunk dfs
    df = pd.concat(df_chunks)
    del df_chunks

    states = df.StateAbbr.unique()

    collapsed_states = []

    for state in tqdm(states):
        filtered_df = df.query("StateAbbr == @state")

        # Compute the max and min speeds at zcta level

        filtered_df["is_advanced"] = filtered_df["TechCode"].apply(
            lambda x: True if x in ([42, 43, 50]) else False
        )

        # Group by ZCTA code and compute the average speeds
        average_speeds_block = (
            filtered_df.groupby("BlockCode")[["MaxAdDown", "MaxAdUp"]]
            .mean()
            .reset_index()
        )

        # Count the number of providers per ZCTA code
        provider_count_block = (
            filtered_df.groupby("BlockCode")["Provider_Id"].nunique().reset_index()
        )

        # Merge the results into a single DataFrame
        block_df = pd.merge(average_speeds_block, provider_count_block, on="BlockCode")

        # Group by 'BlockCode' and 'is_advanced', then count occurrences
        count_advanced_tech = filtered_df.groupby(["BlockCode", "is_advanced"])[
            "is_advanced"
        ].count()

        # Use unstack to pivot the data (do not compute)
        coun_advanced_tech = count_advanced_tech.unstack(fill_value=0)

        # Rename the columns
        coun_advanced_tech.columns = ["count_legacy_tech", "count_advanced_tech"]

        final_block_df = pd.merge(block_df, coun_advanced_tech, on="BlockCode")

        final_block_df.to_csv(f"{file_path}{state}.csv", index=False)

        collapsed_states.append(final_block_df)

    gc.collect()

    # concat the state dfs
    df_coverage = pd.concat(collapsed_states)

    df_coverage.to_csv(
        "/content/drive/MyDrive/summer-2023/raw_data/broadband/broadband/fcc_coverage.csv",
        index=False,
    )

### FCC Broadband coverage per ZCTA


In [39]:
census_relational_block = pd.read_csv(
    "/content/drive/MyDrive/summer-2023/raw_data/broadband/broadband/tab20_zcta520_tabblock20_natl.txt",
    sep="|",
)

In [40]:
merged_df = df_coverage.merge(
    census_relational_block[["GEOID_ZCTA5_20", "GEOID_TABBLOCK_20"]],
    left_on="BlockCode",
    right_on="GEOID_TABBLOCK_20",
)

# Group by ZCTA code and compute the average speeds
average_speeds = merged_df.groupby("GEOID_ZCTA5_20")[["MaxAdDown", "MaxAdUp"]].mean()

# Count the number of providers per ZCTA code
provider_count = merged_df.groupby("GEOID_ZCTA5_20")[
    "Provider_Id"
].median()  # take the median of the providers per block

# Sum the type of technologies used per zcta
legacy_adv_df = merged_df.groupby("GEOID_ZCTA5_20").agg(
    {"count_legacy_tech": "sum", "count_advanced_tech": "sum"}
)

# Merge the results into a single DataFrame
broadband_coverage = average_speeds.merge(
    provider_count, left_index=True, right_index=True
)
broadband_coverage = broadband_coverage.merge(
    legacy_adv_df, left_index=True, right_index=True
)

# Rename the columns for clarity
broadband_coverage = broadband_coverage.rename(
    columns={
        "MaxAdDown": "AvgMaxAdDown",
        "MaxAdUp": "AvgMaxAdUp",
        "Provider_Id": "ProviderCount",
    }
).reset_index()

broadband_coverage.rename(columns={"GEOID_ZCTA5_20": "zcta"}, inplace=True)
broadband_coverage.columns = [col.lower() for col in broadband_coverage.columns]

In [41]:
# crosswalk broadband zctas with state and county zctas

df_coverage_merged = broadband_coverage.merge(
    df_st_cd_zcta[["zcta", "cd116", "state"]], on="zcta"
)

In [42]:
columns_speed = ["avgmaxaddown", "avgmaxadup", "providercount"]
tech_type_cols = ["count_legacy_tech", "count_advanced_tech"]

supply_per_cd = df_coverage_merged.groupby(["state", "cd116"]).agg(
    {
        "count_legacy_tech": "sum",
        "count_advanced_tech": "sum",
        "avgmaxaddown": "median",
        "avgmaxadup": "median",
        "providercount": "median",
    }
)
supply_per_st = df_coverage_merged.groupby(["state"]).agg(
    {
        "count_legacy_tech": "sum",
        "count_advanced_tech": "sum",
        "avgmaxaddown": "median",
        "avgmaxadup": "median",
        "providercount": "median",
    }
)

supply_per_cd = supply_per_cd.reset_index()
supply_per_cd["percentage_adv_tech (cd)"] = (
    supply_per_cd.count_advanced_tech / supply_per_cd.count_legacy_tech
)
supply_per_cd.rename(
    columns=dict(
        zip(
            columns_speed,
            ["avgmaxaddown (cd)", "avgmaxadup (cd)", "providercount (cd)"],
        )
    ),
    inplace=True,
)
supply_per_cd.drop(["count_legacy_tech", "count_advanced_tech"], inplace=True, axis=1)

supply_per_st = supply_per_st.reset_index()
supply_per_st.rename(
    columns=dict(
        zip(
            columns_speed,
            ["avgmaxaddown (state)", "avgmaxadup (state)", "providercount (state)"],
        )
    ),
    inplace=True,
)
supply_per_st["percentage_adv_tech (state)"] = (
    supply_per_st.count_advanced_tech / supply_per_st.count_legacy_tech
)
supply_per_st.drop(["count_legacy_tech", "count_advanced_tech"], inplace=True, axis=1)


broadband_supply_st_cd = supply_per_cd.merge(supply_per_st, on=["state"])
broadband_supply_st_cd

,state,cd116,avgmaxaddown (cd),avgmaxadup (cd),providercount (cd),percentage_adv_tech (cd),avgmaxaddown (state),avgmaxadup (state),providercount (state),percentage_adv_tech (state)
0,AK,0,16.200980,2.000000,3.0,0.168395,16.200980,2.000000,3.0,0.168395
1,AL,1,107.173896,7.334624,5.0,0.210623,132.691129,14.466551,5.0,0.217390
2,AL,2,135.292337,35.540725,5.0,0.247517,132.691129,14.466551,5.0,0.217390
3,AL,3,104.041008,9.128911,5.0,0.196259,132.691129,14.466551,5.0,0.217390
4,AL,4,154.662588,24.626614,5.0,0.178469,132.691129,14.466551,5.0,0.217390
...,...,...,...,...,...,...,...,...,...,...
432,WI,8,80.758811,6.068235,7.0,0.127850,109.387198,8.960717,7.0,0.136951
433,WV,1,73.568353,4.458254,6.0,0.176033,81.383903,4.871754,5.0,0.157304
434,WV,2,46.248254,4.467450,5.0,0.135151,81.383903,4.871754,5.0,0.157304
435,WV,3,96.925546,5.483965,5.0,0.159945,81.383903,4.871754,5.0,0.157304


In [43]:
df_broadband_supply_demand = broadband_supply_st_cd.merge(
    broadband_demand, on=["state", "cd116"]
)
df_broadband_supply_demand

,state,cd116,avgmaxaddown (cd),avgmaxadup (cd),providercount (cd),percentage_adv_tech (cd),avgmaxaddown (state),avgmaxadup (state),providercount (state),percentage_adv_tech (state),...,total_state_population,state_code,current_dollar_gdp_2021,current_dollar_gdp_2022,real_gdp_2021,real_gdp_2022,2022_nominal_gdp_per_capita,2022_real_gdp_per_capita,2021_nominal_gdp_per_capita,2021_real_gdp_per_capita
0,AK,0,16.200980,2.000000,3.0,0.168395,16.200980,2.000000,3.0,0.168395,...,732158.0,AK,5.734900e+10,6.361800e+10,5.086900e+10,4.963400e+10,86891.0,67791.0,78329.0,69478.0
1,AL,1,107.173896,7.334624,5.0,0.210623,132.691129,14.466551,5.0,0.217390,...,5024217.0,AL,2.541100e+11,2.778170e+11,2.099790e+11,2.132650e+11,55296.0,42447.0,50577.0,41793.0
2,AL,2,135.292337,35.540725,5.0,0.247517,132.691129,14.466551,5.0,0.217390,...,5024217.0,AL,2.541100e+11,2.778170e+11,2.099790e+11,2.132650e+11,55296.0,42447.0,50577.0,41793.0
3,AL,3,104.041008,9.128911,5.0,0.196259,132.691129,14.466551,5.0,0.217390,...,5024217.0,AL,2.541100e+11,2.778170e+11,2.099790e+11,2.132650e+11,55296.0,42447.0,50577.0,41793.0
4,AL,4,154.662588,24.626614,5.0,0.178469,132.691129,14.466551,5.0,0.217390,...,5024217.0,AL,2.541100e+11,2.778170e+11,2.099790e+11,2.132650e+11,55296.0,42447.0,50577.0,41793.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
432,WI,8,80.758811,6.068235,7.0,0.127850,109.387198,8.960717,7.0,0.136951,...,5893723.0,WI,3.686110e+11,4.017920e+11,3.064670e+11,3.117020e+11,68173.0,52887.0,62543.0,51999.0
433,WV,1,73.568353,4.458254,6.0,0.176033,81.383903,4.871754,5.0,0.157304,...,1793392.0,WV,8.543400e+10,9.558800e+10,7.134300e+10,7.165200e+10,53300.0,39953.0,47638.0,39781.0
434,WV,2,46.248254,4.467450,5.0,0.135151,81.383903,4.871754,5.0,0.157304,...,1793392.0,WV,8.543400e+10,9.558800e+10,7.134300e+10,7.165200e+10,53300.0,39953.0,47638.0,39781.0
435,WV,3,96.925546,5.483965,5.0,0.159945,81.383903,4.871754,5.0,0.157304,...,1793392.0,WV,8.543400e+10,9.558800e+10,7.134300e+10,7.165200e+10,53300.0,39953.0,47638.0,39781.0


In [44]:
df_broadband_supply_demand.to_csv(
    "/content/drive/MyDrive/summer-2023/raw_data/broadband/processed_data/broadband_demand_supply_v2.csv",
    index=False,
)